In [1]:
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator
import cartopy.crs as ccrs               
import cartopy.feature as cfeature         
import cartopy.util as cutil
import xarray as xr                        
import numpy as np 
import pandas as pd
import plotting_module
import xesmf as xe

In [2]:
ozone_dataset = xr.open_dataset("/glade/campaign/acom/acom-climate/UTLS/shawnh/archive/FCnudged_f09.mam.mar27.2000_2021.002/atm/proc/tseries/month_1/FCnudged_f09.mam.mar27.2000_2021.002.cam.h0.O3.200201-202412.nc")
ps_dataset = xr.open_dataset('/glade/campaign/acom/acom-climate/UTLS/shawnh/archive/FCnudged_f09.mam.mar27.2000_2021.002/atm/proc/tseries/month_1/FCnudged_f09.mam.mar27.2000_2021.002.cam.h0.PS.200201-202412.nc')

In [3]:
ozone = ozone_dataset["O3"]

In [4]:
p0 = ozone_dataset["P0"]
hyai = ozone_dataset["hyai"]
hybi = ozone_dataset["hybi"]
ps = ps_dataset['PS']
lev = ozone_dataset.coords['lev']
num_lev = lev.shape[0]

In [5]:
# convert to hPa from Pa
p0 = p0.copy() / 100
ps = ps.copy() / 100

Create climatology

In [6]:
start_date = '2005-02-01'
end_date = '2025-01-01'

# group the 240 month dates based on calendar months for both PDELDRY and O3 variables

truncated_ozone = ozone.sel(time=slice(start_date, end_date))
truncated_ozone = truncated_ozone.transpose('lev','time','lat','lon')

truncated_ps = ps.sel(time=slice(start_date, end_date))
truncated_ps = truncated_ps.transpose('time','lat','lon')


#ozone_monthly_mean = truncated_ozone.groupby('time.month').mean('time')

Tropospheric column calculations

Calculating pressure at hybrid levels

p(k) = a(k) * p0 + b(k) * ps

In [7]:
# threshold value in hPa
threshold = 100

# constants / conversion factor
NAv = 6.0221415e+23                       # molecules in mole
g = 9.81                                  # gravity
MWair = 28.94                             # g/mol
xp_const = (NAv * 10)/(MWair*g)           # scaling factor, pa to hPa and cm to m
DU_CONVERSION = 2.69 * 10**16

In [8]:
# Initialize pressure edge arrays
mod_press_edge_top = xr.zeros_like(truncated_ozone)
mod_press_edge_bottom = xr.zeros_like(truncated_ozone)

In [9]:
# Calculate pressure edge arrays
# Indices start at the top and end at the bottom
for i in range(num_lev):
    mod_press_edge_top[i,:,:,:] = hyai[i]*p0 + hybi[i]*truncated_ps
    mod_press_edge_bottom[i,:,:,:] = hyai[i+1]*p0 + hybi[i+1]*truncated_ps

In [10]:
#print(mod_press_edge_bottom.sel(lat=slice(40,40.5),lon=slice(150,150.5), month=1).values)

In [11]:
filtered_300hpa_upper = mod_press_edge_top.where(mod_press_edge_top >= threshold, drop=False)
filtered_300hpa_lower = mod_press_edge_bottom.where(mod_press_edge_bottom >= threshold, drop=False)

In [12]:
filtered_deltap = filtered_300hpa_lower - filtered_300hpa_upper

pres_top_v_300 = mod_press_edge_top - threshold
pres_bottom_v_300 = mod_press_edge_bottom - threshold

In [13]:
filtered_deltap_sliver = pres_bottom_v_300
filtered_deltap_sliver = filtered_deltap_sliver.where(pres_bottom_v_300 >= 0)
filtered_deltap_sliver = filtered_deltap_sliver.where(pres_top_v_300 < 0)

In [14]:
filtered_deltap_sliver = filtered_deltap_sliver.fillna(0)

In [15]:
# sum up both the layer edge and the pressure column to the ground

combined_filtered_deltap = filtered_deltap.copy().fillna(0) + filtered_deltap_sliver
#combined_filtered_deltap

In [16]:
ozone_array = xr.zeros_like(truncated_ozone)
ozone_array = truncated_ozone.where(combined_filtered_deltap > 0)

In [17]:
ozone_array = ozone_array.fillna(0)

In [18]:
ozone_column = xr.dot(combined_filtered_deltap, xp_const*ozone_array, dims='lev')
ozone_full_du_column = ozone_column.copy() / DU_CONVERSION

In [19]:
ozone_full_du_column.mean(dim={'time','lat','lon'}, skipna=True)

<xarray.DataArray 'O3' ()> Size: 4B
array(64.07781, dtype=float32)

In [22]:
ozone_full_du_column

<xarray.DataArray 'O3' (time: 240, lat: 192, lon: 288)> Size: 53MB
array([[[ 74.15381 ,  74.15381 ,  74.15381 , ...,  74.15381 ,
          74.15381 ,  74.15381 ],
        [ 75.16588 ,  75.16294 ,  75.15836 , ...,  75.17467 ,
          75.172066,  75.168625],
        [ 75.39407 ,  75.37021 ,  75.3431  , ...,  75.46537 ,
          75.44123 ,  75.417244],
        ...,
        [ 96.92319 ,  96.914986,  96.903336, ...,  96.94726 ,
          96.93783 ,  96.93036 ],
        [ 98.03498 ,  98.025505,  98.01363 , ...,  98.06284 ,
          98.05261 ,  98.04357 ],
        [ 99.90871 ,  99.90871 ,  99.90871 , ...,  99.90871 ,
          99.90871 ,  99.90871 ]],

       [[ 72.02343 ,  72.02343 ,  72.02343 , ...,  72.02343 ,
          72.02343 ,  72.02343 ],
        [ 72.27833 ,  72.28405 ,  72.288734, ...,  72.26589 ,
          72.26909 ,  72.27284 ],
        [ 72.67828 ,  72.682816,  72.6875  , ...,  72.660965,
          72.66612 ,  72.67236 ],
...
        [101.045654, 101.0438  , 101.04251 , ..., 101.05664 ,
         101.05329 , 101.04929 ],
        [101.566185, 101.55802 , 101.550545, ..., 101.59358 ,
         101.585014, 101.57607 ],
        [101.825035, 101.825035, 101.825035, ..., 101.825035,
         101.825035, 101.825035]],

       [[ 76.418564,  76.418564,  76.418564, ...,  76.418564,
          76.418564,  76.418564],
        [ 76.71294 ,  76.6978  ,  76.682556, ...,  76.760956,
          76.74536 ,  76.72869 ],
        [ 77.024666,  76.97744 ,  76.93002 , ...,  77.16621 ,
          77.1202  ,  77.07281 ],
        ...,
        [ 91.57534 ,  91.57843 ,  91.580795, ...,  91.56229 ,
          91.56746 ,  91.57171 ],
        [ 91.11824 ,  91.122246,  91.12562 , ...,  91.10388 ,
          91.108536,  91.11349 ],
        [ 90.61153 ,  90.61153 ,  90.61153 , ...,  90.61153 ,
          90.61153 ,  90.61153 ]]], dtype=float32)
Coordinates:
  * lat      (lat) float64 2kB -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon      (lon) float64 2kB 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * time     (time) datetime64[ns] 2kB 2005-02-01 2005-03-01 ... 2025-01-01

In [23]:
ozone_full_du_column.to_netcdf(path="/glade/u/home/mvoncyga/SOARS_2025/100hPa_tco_monthly.nc", format="NETCDF4")

In [21]:
e

NameError: name 'e' is not defined

Regridding and plotting

In [ ]:
omi_mls_ds = xr.open_dataarray("/glade/u/home/mvoncyga/SOARS_2025/OMIMLS_300hpa_monthly_mean_2005_2024_full.nc")
omi_mls_ds = omi_mls_ds.rename({'latitude': 'lat', 'longitude': 'lon'})

# shifting lon to be 0-360
omi_mls_ds['lon'] = omi_mls_ds['lon'] % 360
omi_mls_ds = omi_mls_ds.sortby('lon')

In [ ]:
regridder = xe.Regridder(ozone_full_du_column, omi_mls_ds, 'bilinear', periodic=True)  
cesm_regridded = regridder(ozone_full_du_column)

In [ ]:
from datetime import datetime 

dataset = xr.open_dataset("tco300_omimls_oct2004_to_dec2024_1by1.nc")
ozone = dataset["TropoO3"]
cleaned_ozone_dataset = ozone.where(ozone >= 0)
dataset["TropoO3"] = cleaned_ozone_dataset

ozone = cleaned_ozone_dataset 

# adding month from attributes to coordinate
month = ozone.attrs['Month'].split(',')
first_month = month[0]                              # October 2004
last_month = month[-1].replace(' ', '')             # December 2024

start_date = datetime.strptime(first_month, "%b%Y")
end_date = datetime.strptime(last_month, "%b%Y")

time_coords = pd.date_range(start=start_date, end=end_date, freq='MS')

# renaming and assigning/removing dimensions
ozone = ozone.assign_coords(time=('months', time_coords))
ozone = ozone.rename({"time": "yearmonth",'y_elements': 'latitude', 'x_elements': 'longitude'})
ozone = ozone.swap_dims({'months': 'yearmonth'})

ozone_subsetted = ozone.sel(yearmonth=slice("2005-01-01", "2024-12-01")) ####

seasonal_omi_mls_mean = ozone_subsetted

# adding lat/lon from attributes to coordinates
lat_coords = ozone_subsetted.attrs['latitude'].split(',')
first_lat = float(lat_coords[0])
last_lat = float(lat_coords[-1])

lon_coords = ozone_subsetted.attrs['longitude'].split(',')
first_lon = float(lon_coords[0])
last_lon = float(lon_coords[-1])

lon_coords = np.arange(first_lon, 180.5, 1)
lat_coords = np.arange(first_lat,90.5, 1)

seasonal_omi_mls_mean = seasonal_omi_mls_mean.assign_coords(
    longitude=('longitude', lon_coords),
    latitude=('latitude', lat_coords)
)
seasonal_omi_mls_mean

In [ ]:
seasonal_omi_mls_mean = seasonal_omi_mls_mean.transpose('yearmonth', 'longitude', 'latitude')
ozone_du_column_seasonal_regridded = cesm_regridded.transpose('time','lon','lat')

In [ ]:
bins = np.arange(-89.5, 90.5, step=4)

binned_seasonal_omi_mls_mean = seasonal_omi_mls_mean.groupby_bins('latitude', bins).mean()
binned_seasonal_cesm_regridded = ozone_du_column_seasonal_regridded.groupby_bins('lat', bins).mean()

In [ ]:
binned_seasonal_cesm_regridded

In [ ]:
#binned_seasonal_omi_mls_mean = binned_seasonal_omi_mls_mean.groupby('yearmonth.season').mean('yearmonth')
binned_seasonal_cesm_regridded = binned_seasonal_cesm_regridded.groupby('time.season').mean('time')

In [ ]:
rmse_omi_mls = np.sqrt(((binned_seasonal_omi_mls_mean - binned_seasonal_omi_mls_mean.mean(axis=0))**2).mean(axis=1))
rmse_cesm = np.sqrt(((binned_seasonal_cesm_regridded - binned_seasonal_cesm_regridded.mean(axis=0))**2).mean(axis=1))

In [ ]:
# average out by longitude
binned_seasonal_omi_mls_mean = binned_seasonal_omi_mls_mean.mean('longitude')
binned_seasonal_cesm_regridded = binned_seasonal_cesm_regridded.mean('lon')

In [ ]:
latitudes = np.arange(-89.5, 84.5, step=4)
seasonal_list = ['DJF', 'JJA', 'MAM', 'SON']

In [ ]:
fig, ax = plt.subplots(ncols=2, nrows=2, figsize=(18,12))

for axis in ['top','bottom','left','right']:
    for i in range(2):
        for j in range(2):
            ax[i][j].spines[axis].set_linewidth(1.5)

for i in range(2):
        for j in range(2):
                ax[i][j].spines[axis].set_linewidth(1.5)
                ax[i][j].set_ylabel("Tropospheric $\mathregular{O_3}$ Concentration (DU)", fontsize=15)
                ax[i][j].set_xlabel("Latitude (°)", fontsize=15)
                ax[i][j].set_ylim(0,45)
                ax[i][j].set_xticks([-90,-60,-30,0,30,60,90])
                ax[i][j].xaxis.set_minor_locator(AutoMinorLocator())
                ax[i][j].yaxis.set_minor_locator(AutoMinorLocator())

i=1
ax[0][0].set_title(str(seasonal_list[0]), fontsize=18, loc='left')
ax[0][0].errorbar(latitudes, binned_seasonal_omi_mls_mean[0], yerr=rmse_omi_mls[0], elinewidth=0.75, ecolor='#eb3434', fmt='none', capsize=5, errorevery=i)
ax[0][0].errorbar(latitudes, binned_seasonal_cesm_regridded[0], yerr=rmse_cesm[0], elinewidth=0.75, ecolor='#2fd4f5', fmt='none', capsize=5, errorevery=i)
ax[0][0].plot(latitudes, binned_seasonal_omi_mls_mean[0], color='#eb3434', linewidth=2.5)
ax[0][0].plot(latitudes, binned_seasonal_cesm_regridded[0], color='#2fd4f5', linewidth=2.5)
ax[0][0].fill_between(latitudes, binned_seasonal_omi_mls_mean[0]-rmse_omi_mls[0], binned_seasonal_omi_mls_mean[0]+rmse_omi_mls[0], color='#eb3434', alpha=0.2)
ax[0][0].fill_between(latitudes, binned_seasonal_cesm_regridded[0]-rmse_cesm[0], binned_seasonal_cesm_regridded[0]+rmse_cesm[0], color='#2fd4f5', alpha=0.2)


ax[1][0].set_title(str(seasonal_list[1]), fontsize=18, loc='left')
ax[1][0].errorbar(latitudes, binned_seasonal_omi_mls_mean[1], yerr=rmse_omi_mls[1], elinewidth=0.75, ecolor='#eb3434', fmt='none', capsize=5, errorevery=i)
ax[1][0].errorbar(latitudes, binned_seasonal_cesm_regridded[1], yerr=rmse_cesm[1], elinewidth=0.75, ecolor='#2fd4f5', fmt='none', capsize=5, errorevery=i)
ax[1][0].plot(latitudes, binned_seasonal_omi_mls_mean[1], color='#eb3434', linewidth=2.5)
ax[1][0].plot(latitudes, binned_seasonal_cesm_regridded[1], color='#2fd4f5', linewidth=2.5)
ax[1][0].fill_between(latitudes, binned_seasonal_omi_mls_mean[1]-rmse_omi_mls[1], binned_seasonal_omi_mls_mean[1]+rmse_omi_mls[1], color='#eb3434', alpha=0.2)
ax[1][0].fill_between(latitudes, binned_seasonal_cesm_regridded[1]-rmse_cesm[1], binned_seasonal_cesm_regridded[1]+rmse_cesm[1], color='#2fd4f5', alpha=0.2)


ax[0][1].set_title(str(seasonal_list[2]), fontsize=18, loc='left')
ax[0][1].errorbar(latitudes, binned_seasonal_omi_mls_mean[2], yerr=rmse_omi_mls[2], elinewidth=0.75, ecolor='#eb3434', fmt='none', capsize=5, errorevery=i)
ax[0][1].errorbar(latitudes, binned_seasonal_cesm_regridded[2], yerr=rmse_cesm[2], elinewidth=0.75, ecolor='#2fd4f5', fmt='none', capsize=5, errorevery=i)
ax[0][1].plot(latitudes, binned_seasonal_omi_mls_mean[2], color='#eb3434', linewidth=2.5)
ax[0][1].plot(latitudes, binned_seasonal_cesm_regridded[2], color='#2fd4f5', linewidth=2.5)
ax[0][1].fill_between(latitudes, binned_seasonal_omi_mls_mean[2]-rmse_omi_mls[2], binned_seasonal_omi_mls_mean[2]+rmse_omi_mls[2], color='#eb3434', alpha=0.2)
ax[0][1].fill_between(latitudes, binned_seasonal_cesm_regridded[2]-rmse_cesm[2], binned_seasonal_cesm_regridded[2]+rmse_cesm[2], color='#2fd4f5', alpha=0.2)


ax[1][1].set_title(str(seasonal_list[3]), fontsize=18, loc='left')
ax[1][1].errorbar(latitudes, binned_seasonal_omi_mls_mean[3], yerr=rmse_omi_mls[3], elinewidth=0.75, ecolor='#eb3434', fmt='none', capsize=5, errorevery=i)
ax[1][1].errorbar(latitudes, binned_seasonal_cesm_regridded[3], yerr=rmse_cesm[3], elinewidth=0.75, ecolor='#2fd4f5', fmt='none', capsize=5, errorevery=i)
ax[1][1].plot(latitudes, binned_seasonal_omi_mls_mean[3], color='#eb3434', linewidth=2.5)
ax[1][1].plot(latitudes, binned_seasonal_cesm_regridded[3], color='#2fd4f5', linewidth=2.5)
ax[1][1].fill_between(latitudes, binned_seasonal_omi_mls_mean[3]-rmse_omi_mls[3], binned_seasonal_omi_mls_mean[3]+rmse_omi_mls[3], color='#eb3434', alpha=0.2)
ax[1][1].fill_between(latitudes, binned_seasonal_cesm_regridded[3]-rmse_cesm[3], binned_seasonal_cesm_regridded[3]+rmse_cesm[3], color='#2fd4f5', alpha=0.2)

fig.suptitle('Latitude Comparison', fontsize=25)
plt.legend(['OMI/MLS', 'CESM'], loc='lower right', prop={'size':  15})

#plt.savefig("figures/Latitude_Comparison")
plt.show()

In [ ]:
binned_seasonal_cesm_regridded[0].mean()